#Statistical analysis and model checking with R and Julia
# Part I: Implementation in R

Recently, I decided to re-analyze some old data using increased knowledge of statistical analysis and model calls. But how do I know that if my results make any sense? One thing I have been recently doing is checking my model results using the `GLM` and `MixedModels` packages in Julia. Previously as a postdoc, since everyone else used JMP or SPSS, it would be independently done in those packages and often, not very well. While the results were mostly the same, I doubt there is much confidence when fitting very complicated model calls, such as when there were random slopes incorporated (JMP is not so good at this; SPSS is better).  

So I will re-analyze several old datasets (two published) that might benefit:
- 'Circlipses' (already done in R and published)
- M100 analysis (already done in R and published)
- Psychophysical vowel perception (revisiting dissertation data; done in R; working on publishing revisions)

## 'Circlipses' (audiovisual integration)
Since this has already been demonstrated (github.com/julian3rd/circlipses-revisited), we will skip right to the initial model calls.

In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
library(readr)

data.path <- 
  'https://raw.githubusercontent.com/julian3rd/model-comparison-r-julia/master/data/circlipse-pwr-data-final-2015.csv'

circlipse.pwr.data <- read_csv(data.path, col_names = TRUE)

str(circlipse.pwr.data)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	672 obs. of  7 variables:
 $ Subject: chr  "r1532" "r1532" "r1532" "r1532" ...
 $ Hem    : chr  "1-lh" "1-lh" "1-lh" "1-lh" ...
 $ Harm   : chr  "1-Fm" "1-Fm" "1-Fm" "1-Fm" ...
 $ Cond   : chr  "1-Uni" "1-Uni" "1-Uni" "2-Zero" ...
 $ Area   : chr  "1-AntTem" "2-PosTem" "3-Occ" "1-AntTem" ...
 $ LinPwr : num  4.31e+15 2.89e+15 2.97e+18 4.37e+16 2.06e+17 ...
 $ dBPwr  : num  156 155 185 166 173 ...


The below wound up being the most parsimonious model.

In [3]:
%%R 
library(lme4)
interact.model.parsim <- lmer(dBPwr ~ Hem + Harm + Cond + Area + Hem:Area + Harm:Area + Cond:Area +
                             (Hem + Harm + Cond | Subject), data = circlipse.pwr.data, REML = FALSE)
print(summary(interact.model.parsim))

Loading required package: Matrix

Attaching package: ‘Matrix’

The following objects are masked from ‘package:base’:

    crossprod, tcrossprod

Loading required package: Rcpp
Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: dBPwr ~ Hem + Harm + Cond + Area + Hem:Area + Harm:Area + Cond:Area +  
    (Hem + Harm + Cond | Subject)
   Data: circlipse.pwr.data

     AIC      BIC   logLik deviance df.resid 
  4717.2   4897.6  -2318.6   4637.2      632 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.0091 -0.5682  0.0702  0.6692  2.7253 

Random effects:
 Groups   Name             Variance Std.Dev. Corr                         
 Subject  (Intercept)      19.579   4.425                                 
          Hem2-rh           6.687   2.586    -0.22                        
          Harm2-SecondHarm 10.954   3.310    -0.74 -0.19                  
          Cond2-Zero        4.028   2.007     0.35 -0.26  0.15            
          Cond3-HalfPi      2.546   

## M100 analysis in 16p11.2 deletion and duplication
This is data from a recently published study examining M100 latency in typically-developing children, children with 16p11.2 deletions and 16p11.2 duplications (http://cercor.oxfordjournals.org/content/early/2015/02/11/cercor.bhv008.short?rss=1). The end result was that children with 16p11.2 deletions exhibited delayed M100 latencies relative to controls; there was no significant difference between 16p11.2 duplications and controls. These data have not only fixed effects but also a continuous covariate of Age that must be accounted for.

In [4]:
%%R

data.path <- 
  'https://raw.githubusercontent.com/julian3rd/model-comparison-r-julia/master/data/simons-child-data-final.csv'

child.data <- 
  read_csv(data.path, col_names = TRUE, col_types = c('ccdcccd'))
str(child.data)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	792 obs. of  7 variables:
 $ Subject    : chr  "3002-102" "3002-102" "3002-102" "3002-102" ...
 $ Site       : chr  "UCSF" "UCSF" "UCSF" "UCSF" ...
 $ Age_Calc   : num  10.8 10.8 10.8 10.8 10.8 ...
 $ Case       : chr  "duplication" "duplication" "duplication" "duplication" ...
 $ Hem        : chr  "1-LH" "1-LH" "1-LH" "1-LH" ...
 $ Cond       : chr  "1-200" "2-300" "3-500" "4-1000" ...
 $ M100LatCorr: num  NA NA NA NA 134 120 118 138 152 148 ...


In [5]:
%%R

child.addmodel <- 
  lmer(M100LatCorr ~ Hem + Cond + Case + Age_Calc + Site + (Cond + Hem|Subject),
       data = child.data, REML = F)

In [6]:
%R print(summary(child.addmodel))

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: M100LatCorr ~ Hem + Cond + Case + Age_Calc + Site + (Cond + Hem |  
    Subject)
   Data: child.data

     AIC      BIC   logLik deviance df.resid 
  4253.0   4360.0  -2101.5   4203.0      509 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.6691 -0.4596 -0.0101  0.4325  2.6082 

Random effects:
 Groups   Name        Variance Std.Dev. Corr                   
 Subject  (Intercept) 479.23   21.891                          
          Cond2-300    13.28    3.644    0.32                  
          Cond3-500    41.98    6.479   -0.09  0.66            
          Cond4-1000   75.08    8.665   -0.31  0.61  0.89      
          Hem2-RH     364.62   19.095   -0.51 -0.13  0.22  0.00
 Residual              49.67    7.048                          
Number of obs: 534, groups:  Subject, 96

Fixed effects:
                Estimate Std. Error t value
(Intercept)     192.3710    10.4107  18.478
Hem2-RH         -14.5908   

## Proportion correct analysis in AX task
These data are taken from an experiment from my dissertation (still trying to get published) that involved observers indicating whether or not they perceived a pair of signals being identical or different. Responses (1 = correct, 0 = incorrect) and RT were recorded; only correct/incorrect responses will be analyzed. The 'real' analysis involves using an GLMM with a binomial errors (logit link function). However, since I cannot implement this in Julia just yet I will take the GLM approach and not include the random effect terms.

In [7]:
%%R

data.path <- 
  'https://raw.githubusercontent.com/julian3rd/model-comparison-r-julia/master/data/psycho-data-frame.csv'

psycho.data <- read_csv(data.path, col_names = TRUE, col_types = c('ccccccdddd'))

str(psycho.data)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	12800 obs. of  10 variables:
 $ SubjectAssgn: chr  "a-i-subject-1" "a-i-subject-1" "a-i-subject-1" "a-i-subject-1" ...
 $ GenID       : chr  "Subject1" "Subject1" "Subject1" "Subject1" ...
 $ ExpAssgn    : chr  "a-i" "a-i" "a-i" "a-i" ...
 $ Vowel       : chr  "a" "a" "a" "a" ...
 $ Item        : chr  "a-2-2" "a-2-2" "a-2-2" "a-2-2" ...
 $ ItemType    : chr  "same" "same" "same" "same" ...
 $ RespNum     : num  1 1 1 1 1 1 1 1 1 1 ...
 $ RespNum2    : num  100 100 100 100 100 100 100 100 100 100 ...
 $ absRT       : num  685 707 443 561 535 ...
 $ logAbsRT    : num  2.84 2.85 2.65 2.75 2.73 ...


In [8]:
%%R

#generate pair levels (simplifies plot and analysis)
psycho.data$ItemPair <- 
  gl( n = 10, k = 20,
      labels = c('2-2','23-2','23-23','23-4','23-square','2-square','4-2', '4-4','4-square','square-square'))

In [9]:
%%R

psycho.vowel.glm <- glm(RespNum ~ Vowel, binomial, data = psycho.data)

psycho.item.pair.glm <- glm(RespNum ~ ItemPair, binomial, data = psycho.data)

psycho.vowel.item.glm <-  glm(RespNum ~ Vowel * ItemType, binomial, data = psycho.data)

psycho.item.glm <- glm(RespNum ~ Item, binomial, data = psycho.data)

In [10]:
%R print(summary(psycho.vowel.glm))


Call:
glm(formula = RespNum ~ Vowel, family = binomial, data = psycho.data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8011  -1.2060   0.6634   1.1491   1.1491  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.06690    0.02501   2.675  0.00748 ** 
Voweli       1.33509    0.05096  26.197  < 2e-16 ***
Vowelu       0.84983    0.04645  18.297  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 16763  on 12799  degrees of freedom
Residual deviance: 15874  on 12797  degrees of freedom
AIC: 15880

Number of Fisher Scoring iterations: 4



In [11]:
%R print(summary(psycho.item.pair.glm))


Call:
glm(formula = RespNum ~ ItemPair, family = binomial, data = psycho.data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2537  -0.9165   0.4319   0.9657   2.1720  

Coefficients:
                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)            2.32514    0.09813  23.694   <2e-16 ***
ItemPair23-2          -1.88373    0.11362 -16.579   <2e-16 ***
ItemPair23-23         -0.11920    0.13554  -0.879    0.379    
ItemPair23-4          -2.97523    0.11444 -25.998   <2e-16 ***
ItemPair23-square     -1.80430    0.11389 -15.842   <2e-16 ***
ItemPair2-square      -4.58466    0.13696 -33.476   <2e-16 ***
ItemPair4-2           -2.13708    0.11306 -18.902   <2e-16 ***
ItemPair4-4           -0.17894    0.13403  -1.335    0.182    
ItemPair4-square      -2.21565    0.11298 -19.611   <2e-16 ***
ItemPairsquare-square  0.13217    0.14275   0.926    0.355    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for bino

In [12]:
%R print(summary(psycho.vowel.item.glm))


Call:
glm(formula = RespNum ~ Vowel * ItemType, family = binomial, 
    data = psycho.data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.1874  -0.7709   0.4418   0.7885   1.6484  

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)         -1.06146    0.03693  -28.74   <2e-16 ***
Voweli               2.07040    0.06343   32.64   <2e-16 ***
Vowelu               1.40436    0.05924   23.71   <2e-16 ***
ItemTypesame         3.33014    0.07720   43.14   <2e-16 ***
Voweli:ItemTypesame -2.06117    0.13372  -15.41   <2e-16 ***
Vowelu:ItemTypesame -1.37646    0.13232  -10.40   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 16763  on 12799  degrees of freedom
Residual deviance: 12380  on 12794  degrees of freedom
AIC: 12392

Number of Fisher Scoring iterations: 4



In [13]:
%R print(summary(psycho.item.glm))


Call:
glm(formula = RespNum ~ Item, family = binomial, data = psycho.data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3548  -0.5133   0.4359   0.5098   2.4478  

Coefficients:
                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)          2.250e+00  1.346e-01  16.718  < 2e-16 ***
Itema-2-square      -5.195e+00  2.259e-01 -23.001  < 2e-16 ***
Itema-23-2          -2.695e+00  1.571e-01 -17.154  < 2e-16 ***
Itema-23-23         -8.745e-02  1.871e-01  -0.467   0.6402    
Itema-23-4          -4.211e+00  1.804e-01 -23.335  < 2e-16 ***
Itema-23-square     -2.675e+00  1.570e-01 -17.038  < 2e-16 ***
Itema-4-2           -3.046e+00  1.594e-01 -19.108  < 2e-16 ***
Itema-4-4            3.768e-14  1.904e-01   0.000   1.0000    
Itema-4-square      -3.236e+00  1.613e-01 -20.061  < 2e-16 ***
Itema-square-square  1.751e-01  1.976e-01   0.886   0.3757    
Itemi-2-2            1.339e-01  2.420e-01   0.553   0.5801    
Itemi-2-square      -3.758e+00  1.979e-

## Output and conclusion
So we now have tables that give our coefficients, and in some cases exact(-ish) p-values for those coefficients and can get effect sizes from those. The models are also pretty well-behaved. Let's now switch to Julia and see if our results are at least consistent.

In [14]:
import IPython
print 'IPython version: ', IPython.__version__

IPython version:  3.1.0


In [15]:
%R print(sessionInfo())

R version 3.1.3 (2015-03-09)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: OS X 10.9.5 (Mavericks)

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] tools     stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] lme4_1.1-7   Rcpp_0.11.6  Matrix_1.2-0 readr_0.1.0 

loaded via a namespace (and not attached):
[1] curl_0.5        grid_3.1.3      lattice_0.20-31 MASS_7.3-40    
[5] minqa_1.2.4     nlme_3.1-120    nloptr_1.0.4    splines_3.1.3  
